# Issue #10 Problem with Primary Beam at different frequencies

edited by JC, Martin, Louise

The issue seems to be solved, Jean implemented the scaling of the primary beam with the frequency. At 220, we know it is not a good approximation so we print some warnings. 

In [ ]:
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(16,4))
plt.rc('font',size=12)
plt.rc('text',usetex=False)
plt.rc('image', cmap='viridis')

import os

path = os.getcwd()
path_mm = os.path.dirname(path)
import sys
sys.path.append(path_mm)

import numpy as np
import qubic
from qubic.lib.Instrument.Qinstrument import QubicInstrument, QubicMultibandInstrument, QubicMultibandInstrumentTrapezoidalIntegration


In [ ]:
# This is the analytical synthesized for a pixel at the focal plane center 
# without accounting for the primary beam (just the mukltple peaks)
def sb_noprim(th_deg, nu):
    th = np.radians(th_deg)
    lam = 3e8/nu
    P = 20
    deltax = 0.013
    abscissa = np.pi * deltax/lam * th
    sb = np.sin(P*abscissa)**2 / np.sin(abscissa)**2
    return sb/np.max(sb)

# Test Monochromatic

In [ ]:
# With Multiband = False  => using a QubicInstrument
d = qubic.lib.Qdictionary.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['multiband'] = False

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
freqs = np.array([131.25*1e9, 150*1e9, 168.75*1e9])
d['config'] = 'FI'
# freqs = np.array([1.32638868e+11, 1.50398340e+11, 1.67001321e+11])

nn = 1000
thetas = np.linspace(-30, 30, nn)
phis = 0

fig, axs = plt.subplots(3, 1, figsize=(20, 15))
axs = np.ravel(axs)

beams = ['gaussian', 'fitted_beam', 'multi_freq']

for j, ibeam in enumerate(beams):
    print('\n Computing beam model: {}'.format(ibeam))
    allbeams = np.zeros((len(freqs), nn))
    d['beam_shape'] = ibeam
    ax1 = axs[j]
#     ax1 = axs[2*j]
#     ax2 = axs[2*j+1]
    for i in range(len(freqs)):
        d['filter_nu'] = freqs[i]
        q = QubicInstrument(d)
        allbeams[i, :] = q.primary_beam(np.radians(np.abs(thetas)), phis)

        p = ax1.plot(thetas, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
        ax1.plot(thetas, allbeams[i,:]*sb_noprim(thetas, freqs[i]), color=p[0].get_color())
        ax1.set_title(ibeam)#'d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
        ax1.set_xlim(-20, 20)
        ax1.legend()

# Gaussian beam monochromatic

In [ ]:
# With Multiband = False  => using a QubicInstrument
d = qubic.lib.Qdictionary.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['multiband'] = False

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
freqs = np.array([1.36984337e+11, 1.48954079e+11, 1.61969742e+11])
d['config'] = 'FI'

nn = 1000
thetas = np.linspace(-12, -6, nn)
phis = 0

fig, axs = plt.subplots(figsize=(10, 10))
axs = np.ravel(axs)

beams = ['gaussian']

for j, ibeam in enumerate(beams):
    print('\n Computing beam model: {}'.format(ibeam))
    allbeams = np.zeros((len(freqs), nn))
    d['beam_shape'] = ibeam
    ax1 = axs[j]
#     ax1 = axs[2*j]
#     ax2 = axs[2*j+1]
    for i in range(len(freqs)):
        d['filter_nu'] = freqs[i]
        q = QubicInstrument(d)

        print("filter nu : ", q.filter.nu/1e9)
        print("filter bw : ", q.filter.bandwidth/1e9)

        allbeams[i, :] = q.primary_beam(np.radians(np.abs(thetas)), phis)

        p = ax1.plot(thetas, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
        ax1.plot(thetas, allbeams[i,:]*sb_noprim(thetas, freqs[i]), color=p[0].get_color())
        ax1.set_title(ibeam)#'d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
        ax1.set_xlim(-12, -6)
        ax1.legend()
        ind = np.where((allbeams[i]*sb_noprim(thetas, freqs[i])) == np.max(allbeams[i]*sb_noprim(thetas, freqs[i])))[0]
        plt.vlines(thetas[ind], 0, 0.5, color=p[0].get_color())
        plt.hlines(allbeams[i, ind]*sb_noprim(thetas, freqs[i])[ind], -12, -6, color=p[0].get_color())

# Test Polychromatic

In [ ]:
q.primary_beam??

In [ ]:
# With multiband = True code have to be written differently
d = qubic.lib.Qdictionary.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['multiband'] = True

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
d['config'] = 'FI'
d['filter_nu'] = 150e9 
d["detector_nep"] = float(4.7e-17)
d["nf_sub"] = 3

thetas = np.linspace(-30, 30, nn)

q = QubicMultibandInstrument(d)
nsub = len(q)
nus = np.zeros(nsub)
for i in range(nsub): 
    nus[i] = q[i].filter.nu

beams = ['gaussian', 'fitted_beam', 'multi_freq']

fig, axs = plt.subplots(3, 1, figsize=(12, 14))
axs = np.ravel(axs)
print('============== Starting the loop ===================')
for b, ibeam in enumerate(beams):
    print('Computing beam model: {}'.format(ibeam))
    d['beam_shape'] = ibeam
    q = QubicMultibandInstrument(d)
    ax = axs[b]
    for i in range(nsub):
        print(i)
        allbeams[i, :] = q[i].primary_beam(np.radians(np.abs(thetas)),phis)
        p = ax.plot(thetas, allbeams[i, :], '--', 
                    label='nu={0:6.1f}'.format(nus[i]/1e9))
        ax.plot(thetas, allbeams[i, :] * sb_noprim(thetas, freqs[i]), 
                color=p[0].get_color())

        ax.set_title('d[beam_shape]=' + ibeam + ' , Multiband=' + str(d['multiband']))
        ax.set_xlim(-20, 20)
        ax.legend()

In [ ]:
# With multiband = True code have to be written differently
d = qubic.lib.Qdictionary.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['MultiBand'] = True

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
d['config'] = 'FI'
d['filter_nu'] = 150e9 
d["detector_nep"] = float(4.7e-17)
d["nf_sub"] = 6

thetas = np.linspace(-30, 30, nn)

q = QubicMultibandInstrumentTrapezoidalIntegration(d)
nsub = len(q)
nus = np.zeros(nsub)
for i in range(nsub): 
    nus[i] = q[i].filter.nu
print(nus)


beams = ['gaussian', 'fitted_beam', 'multi_freq']

fig, axs = plt.subplots(3, 1, figsize=(12, 14))
axs = np.ravel(axs)
print('============== Starting the loop ===================')
for b, ibeam in enumerate(beams):
    print('Computing beam model: {}'.format(ibeam))
    d['beam_shape'] = ibeam
    q = QubicMultibandInstrument(d)
    ax = axs[b]
    for i in range(int(nsub/2)):
        print(i)
        allbeams[i, :] = q[i].primary_beam(np.radians(np.abs(thetas)),phis)
        p = ax.plot(thetas, allbeams[i, :], '--', 
                    label='nu={0:6.1f}'.format(nus[i]/1e9))
        ax.plot(thetas, allbeams[i, :] * sb_noprim(thetas, freqs[i]), 
                color=p[0].get_color())

        ax.set_title('d[beam_shape]=' + ibeam + ' , Multiband=' + str(d['MultiBand']))
        ax.set_xlim(-20, 20)
        ax.legend()

# Gaussian Beam Polychromatic

In [ ]:
# With multiband = True code have to be written differently
d = qubic.lib.Qdictionary.qubicDict()
d.read_from_file('global_source_oneDet.dict')

d['MultiBand'] = True

# Test with 150 or 220 GHz 
# At 220 you should use FI and not TD
d['config'] = 'FI'
d['filter_nu'] = 150e9 
d["detector_nep"] = float(4.7e-17)
d["nf_sub"] = 6

thetas = np.linspace(-12, -6, nn)

q = QubicMultibandInstrumentTrapezoidalIntegration(d)
nsub = len(q)
nus = np.zeros(nsub)
for i in range(nsub): 
    nus[i] = q[i].filter.nu
print(nus)

allbeams = np.zeros((int(len(nus)/2), nn))

beams = ['gaussian']

fig, axs = plt.subplots(figsize=(10, 10))   
axs = np.ravel(axs)
print('============== Starting the loop ===================')
for b, ibeam in enumerate(beams):
    print('Computing beam model: {}'.format(ibeam))
    d['beam_shape'] = ibeam
    q = QubicMultibandInstrumentTrapezoidalIntegration(d)
    ax = axs[b]
    for i in range(int(nsub/2)):
        print("Filter nu : ", q[i].filter.nu/1e9)
        print('Filter relative bandwidth: ', q[i].filter.bandwidth/1e9)
        allbeams[i, :] = q[i].primary_beam(np.radians(np.abs(thetas)),phis)
        p = ax.plot(thetas, allbeams[i, :], '--', 
                    label='nu={0:6.1f} GHz'.format(nus[i]/1e9))
        ax.plot(thetas, allbeams[i, :] * sb_noprim(thetas, nus[i]), 
                color=p[0].get_color())

        ax.set_title('d[beam_shape]=' + ibeam + ' , Multiband=' + str(d['MultiBand']))
        ax.set_xlim(-12, -6)
        ax.legend()
        ind = np.where((allbeams[i]*sb_noprim(thetas, nus[i])) == np.max(allbeams[i]*sb_noprim(thetas, nus[i])))[0]
        plt.vlines(thetas[ind], 0, 0.5, color=p[0].get_color())
        plt.hlines(allbeams[i, ind]*sb_noprim(thetas, nus[i])[ind], -12, -6, color=p[0].get_color())